## Etapa 2: Engenharia de Features (Avançada)

Neste notebook, vamos carregar os dados consolidados e criar as features (sinais) que o nosso modelo de IA irá usar para aprender a prever a compatibilidade entre um candidato e uma vaga.

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re
import json

print("--- ETAPA 2: ENGENHARIA DE FEATURES (AVANÇADA) ---")

--- ETAPA 2: ENGENHARIA DE FEATURES (AVANÇADA) ---


### Bloco 1: Carregar os dados consolidados do JSON

In [2]:
print("\nCarregando 'dados_consolidados.json'...")
try:
    df = pd.read_json('../data/processed/dados_consolidados.json', lines=True)
    # Insira esta célula para ver os nomes exatos das colunas
    print("--- Lista de Nomes de Colunas Reais no DataFrame ---")
    print(df.columns.tolist())
    print("Dados carregados com sucesso.")
except FileNotFoundError:
    print("Erro: O arquivo 'dados_consolidados.json' não foi encontrado. Execute a Etapa 1 primeiro.")
    exit()


Carregando 'dados_consolidados.json'...
--- Lista de Nomes de Colunas Reais no DataFrame ---
['id_vaga', 'prospect.nome', 'prospect.codigo', 'prospect.situacao_candidado', 'prospect.data_candidatura', 'prospect.ultima_atualizacao', 'prospect.comentario', 'prospect.recrutador', 'informacoes_basicas.data_requicisao', 'informacoes_basicas.limite_esperado_para_contratacao', 'informacoes_basicas.titulo_vaga', 'informacoes_basicas.vaga_sap', 'informacoes_basicas.cliente', 'informacoes_basicas.solicitante_cliente', 'informacoes_basicas.empresa_divisao', 'informacoes_basicas.requisitante', 'informacoes_basicas.analista_responsavel', 'informacoes_basicas.tipo_contratacao', 'informacoes_basicas.prazo_contratacao', 'informacoes_basicas.objetivo_vaga', 'informacoes_basicas.prioridade_vaga', 'informacoes_basicas.origem_vaga', 'informacoes_basicas.superior_imediato', 'informacoes_basicas.nome', 'informacoes_basicas.telefone', 'informacoes_basicas.data_inicial', 'informacoes_basicas.data_final', 'in

### Bloco 2: Preparação dos Textos

In [3]:
if df is not None:
    print("\nLimpando e combinando campos de texto...")
    
    # Usando os nomes de coluna padronizados com prefixo
    cols_vaga = [
        'informacoes_basicas.titulo_vaga', 
        'perfil_vaga.principais_atividades', 
        'perfil_vaga.competencia_tecnicas_e_comportamentais'
    ]
    cols_candidato = [
        'cv_pt', 
        'informacoes_profissionais.conhecimentos_tecnicos', 
        'informacoes_profissionais.area_atuacao'
    ]
    
    for col in cols_vaga + cols_candidato:
        # Garante que a coluna exista antes de tentar preencher NAs
        if col in df.columns:
            df[col] = df[col].fillna('')

    df['texto_vaga'] = df[cols_vaga].agg(' '.join, axis=1).str.lower()
    df['texto_candidato'] = df[cols_candidato].agg(' '.join, axis=1).str.lower()
    print("Campos de texto combinados e normalizados.")


Limpando e combinando campos de texto...
Campos de texto combinados e normalizados.


### Bloco 3: Criação das Features

In [4]:
if df is not None:
    # Feature 1: Similaridade de Texto (NLP)
    print("\n1. Calculando a similaridade de texto com TF-IDF...")
    vectorizer = TfidfVectorizer()
    corpus = df['texto_vaga'].tolist() + df['texto_candidato'].tolist()
    tfidf_matrix = vectorizer.fit_transform(corpus)
    vaga_vectors = tfidf_matrix[0:len(df)]
    candidato_vectors = tfidf_matrix[len(df):]
    df['similaridade_texto'] = [cosine_similarity(vaga_vectors[i], candidato_vectors[i])[0][0] for i in range(len(df))]
    print("   Feature 'similaridade_texto' criada.")

    # Features de Compatibilidade de Idioma e SAP
    print("\n2. Criando features de compatibilidade de Idioma e SAP...")
    mapa_niveis = {'Nenhum': 0, 'Básico': 1, 'Intermediário': 2, 'Técnico': 3, 'Avançado': 4, 'Fluente': 5}
    
    df['vaga_nivel_ingles_num'] = df['perfil_vaga.nivel_ingles'].fillna('Nenhum').apply(lambda x: mapa_niveis.get(x, 0))
    df['candidato_nivel_ingles_num'] = df['formacao_e_idiomas.nivel_ingles'].fillna('Nenhum').apply(lambda x: mapa_niveis.get(x, 0))
    df['match_nivel_ingles'] = (df['candidato_nivel_ingles_num'] >= df['vaga_nivel_ingles_num']).astype(int)
    
    df['vaga_nivel_espanhol_num'] = df['perfil_vaga.nivel_espanhol'].fillna('Nenhum').apply(lambda x: mapa_niveis.get(x, 0))
    df['candidato_nivel_espanhol_num'] = df['formacao_e_idiomas.nivel_espanhol'].fillna('Nenhum').apply(lambda x: mapa_niveis.get(x, 0))
    df['match_nivel_espanhol'] = (df['candidato_nivel_espanhol_num'] >= df['vaga_nivel_espanhol_num']).astype(int)
    
    df['vaga_requer_sap'] = (df['informacoes_basicas.vaga_sap'] == 'Sim').astype(int)
    df['candidato_tem_sap'] = df['texto_candidato'].str.contains('sap', flags=re.IGNORECASE, regex=True).astype(int)
    df['match_sap'] = ((df['vaga_requer_sap'] == 0) | ((df['vaga_requer_sap'] == 1) & (df['candidato_tem_sap'] == 1))).astype(int)
    print("   Features de idioma e SAP criadas.")

    # Feature: Match de Anos de Experiência
    print("\n3. Criando feature de Anos de Experiência...")
    def extract_years_experience(text):
        if not isinstance(text, str): return 0
        matches = re.findall(r'(\d{1,2})\s+an[o|os]', text, re.IGNORECASE)
        years = [int(year) for year in matches]
        return max(years) if years else 0

    df['vaga_anos_exp'] = df['texto_vaga'].apply(extract_years_experience)
    df['candidato_anos_exp'] = df['texto_candidato'].apply(extract_years_experience)
    df['match_anos_experiencia'] = (df['candidato_anos_exp'] >= df['vaga_anos_exp']).astype(int)
    print("   Feature 'match_anos_experiencia' criada.")

    # Feature: Score de Habilidades Técnicas
    print("\n4. Criando feature de Score de Habilidades Técnicas...")
    SKILLS_LIST = ['java', 'python', 'sql', 'oracle', 'aws', 'azure', 'react', 'angular', 'scrum', 'agile', 'devops', 'api', 'typescript', 'javascript', 'docker', 'kubernetes']
    def calculate_skills_match(vaga_text, candidate_text):
        required_skills = {skill for skill in SKILLS_LIST if skill in vaga_text}
        if not required_skills:
            return 1.0 
        matched_skills = sum(1 for skill in required_skills if skill in candidate_text)
        return matched_skills / len(required_skills)

    df['skills_match_score'] = df.apply(lambda row: calculate_skills_match(row['texto_vaga'], row['texto_candidato']), axis=1)
    print("   Feature 'skills_match_score' criada.")

    # Feature: Match de Nível Profissional
    print("\n5. Criando feature de Nível Profissional...")
    mapa_senioridade = {'júnior': 1, 'junior': 1, 'pleno': 2, 'sênior': 3, 'senior': 3, 'especialista': 4}
    def get_seniority_level(text):
        if not isinstance(text, str): return 0
        text = text.lower()
        for level, value in mapa_senioridade.items():
            if level in text:
                return value
        return 0

    df['vaga_nivel_prof'] = df['perfil_vaga.nivel profissional'].fillna('').apply(get_seniority_level)
    df['candidato_nivel_prof'] = df['informacoes_profissionais.nivel_profissional'].fillna('').apply(get_seniority_level)
    df['match_nivel_profissional'] = (df['candidato_nivel_prof'] >= df['vaga_nivel_prof']).astype(int)
    print("   Feature 'match_nivel_profissional' criada.")


1. Calculando a similaridade de texto com TF-IDF...
   Feature 'similaridade_texto' criada.

2. Criando features de compatibilidade de Idioma e SAP...
   Features de idioma e SAP criadas.

3. Criando feature de Anos de Experiência...
   Feature 'match_anos_experiencia' criada.

4. Criando feature de Score de Habilidades Técnicas...
   Feature 'skills_match_score' criada.

5. Criando feature de Nível Profissional...
   Feature 'match_nivel_profissional' criada.


### Bloco Final: Seleção e Salvamento

In [5]:
if df is not None:
    print("\nSelecionando features finais e salvando o resultado...")
    features_selecionadas = [
        'id_vaga',
        'prospect.codigo',
        'match',
        'similaridade_texto',
        'match_nivel_ingles',
        'match_nivel_espanhol',
        'match_sap',
        'match_anos_experiencia',
        'skills_match_score',
        'match_nivel_profissional'
    ]
    df_features = df[features_selecionadas]

    df_features.to_json('../data/processed/dados_com_features.json', orient='records', lines=True, force_ascii=False)
    print("\n--- DataFrame com Novas Features ---")
    display(df_features.head())
    print(f"\nDimensões do DataFrame de features: {df_features.shape}")
    print("\nArquivo 'dados_com_features.json' salvo com sucesso!")
    print("\n--- ETAPA 2 CONCLUÍDA ---")


Selecionando features finais e salvando o resultado...

--- DataFrame com Novas Features ---


,id_vaga,prospect.codigo,match,similaridade_texto,match_nivel_ingles,match_nivel_espanhol,match_sap,match_anos_experiencia,skills_match_score,match_nivel_profissional
0,4530,25529,0,0.141520,1,1,1,1,1.0,0
1,4531,25364,1,0.257011,1,1,1,1,1.0,0
2,4534,26361,0,0.147204,0,1,1,0,1.0,0
3,4534,26205,0,0.148577,0,1,1,1,1.0,0
4,4534,26003,0,0.192539,0,1,1,1,1.0,0



Dimensões do DataFrame de features: (13846, 10)

Arquivo 'dados_com_features.json' salvo com sucesso!

--- ETAPA 2 CONCLUÍDA ---
